In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1. Coleta Inicial de Dados

### Colab

In [ ]:
#file_path = "/content/drive/MyDrive/SI5/electricity-normalized.arff"
#data, meta = arff.loadarff(file_path)

#df = pd.DataFrame(data)

### Local

In [ ]:
file_path = "data/raw/electricity-normalized.arff"
data, meta = arff.loadarff(file_path)

df = pd.DataFrame(data)

### Converter colunas categóricas de bytes para strings


In [ ]:
for col in df.select_dtypes([object]):
    df[col] = df[col].str.decode("utf-8")

## 2. Descrição dos dados

In [ ]:
df.info()

In [ ]:
df['class'] = df['class'].apply(lambda val: 1 if val == 'UP' else 0)

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

## 3. Exploração dos dados Data

### Identificando Outliers

In [ ]:
# Filtra apenas as colunas numéricas
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Calcula Q1, Q3 e IQR para cada coluna numérica
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Detecta outliers
outliers = ((df[numerical_cols] < (Q1 - 1.5 * IQR)) | (df[numerical_cols] > (Q3 + 1.5 * IQR))).sum()
outliers

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df)
plt.title("Boxplot das Variáveis Contínuas")
plt.xticks(rotation=90)
plt.show()

In [ ]:
categorical_columns = df.select_dtypes(include=[object]).columns
categorical_columns

### Distribuição da Classe Alvo

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x=df['class'])
plt.title("Distribuição da Classe Alvo")
plt.show()

### Distribuição das variáveis categóricas


In [ ]:
for col in categorical_columns:
    plt.figure(figsize=(6, 6))
    df[col].value_counts().plot.pie(autopct='%1.1f%%')
    plt.title(f"Distribuição da Variável Categórica: {col}")
    plt.ylabel('')
    plt.show()

### Histogramas para variáveis contínuas


In [ ]:
df.hist(bins=30, figsize=(15, 10))
plt.suptitle("Histogramas das Variáveis Contínuas")
plt.show()

### Gráficos comparativos para distribuição dos dados entre as diferentes classes


In [ ]:
for col in df.select_dtypes(include=[np.number]).columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df['class'], y=df[col])
    plt.title(f"Distribuição da Variável {col} entre as Classes")
    plt.show()

## 4. Verificação da qualidade dos dados

### Matriz de Correlação

In [ ]:
corr = df.corr().iloc[::-1, ::-1]

fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(corr, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(cax)

for (i, j), val in np.ndenumerate(corr):
    ax.text(j, i, f"{val:.2f}", ha='center', va='center', color='black')


ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns)
ax.set_yticklabels(corr.columns)

plt.xticks(rotation=90)

plt.title("Matriz de Correlação", pad=20)
plt.show()

### Redução de Dimensionalidade

In [ ]:
list_features = ['nswprice', 'nswdemand', 'vicprice', 'vicdemand', 'transfer']

In [ ]:
for variavel in list_features:
    precos_maximos_diarios = []
    medicoes_por_dia = 48
    
    for i in range(0, len(df), medicoes_por_dia):
        max_diario = df[variavel][i:i+medicoes_por_dia].max()
        precos_maximos_diarios.append(max_diario)


    dias = np.arange(len(precos_maximos_diarios))

    # Criar o gráfico de dispersão
    plt.figure(figsize=(20, 5))
    plt.scatter(dias, precos_maximos_diarios, color='b', label='Preço Máximo Diário', s=10)

    # Label do gráfico
    plt.xlabel('Dias')
    plt.ylabel('Preço Máximo')
    plt.title(f'Variação da {variavel} Máxima Diária ao Longo do Tempo')

    plt.xticks(rotation=45)
    plt.xlim(min(dias), max(dias))
    plt.legend()
    plt.grid()
    plt.show()

### Box Plot de demanda, valores e transfer ao longo pelos dias da semana

In [ ]:
for col in list_features:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=sorted(df['day']), y=df[col])
    plt.title(f"Distribuição da Variável {col} entre as Classes")
    plt.show()

### Redução de Dimensionalidade

In [ ]:
df['class'] = df['class'].replace({'UP': 1, 'DOWN': 0})
Y_df = df['class']
X_df = df.drop(columns=['class'])

### t-SNE

In [ ]:
tsne_2d = TSNE(n_components=2, random_state=42)
X_tsne = tsne_2d.fit_transform(X_df)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=Y_df, cmap='viridis', s=50)
plt.colorbar()
plt.title("Visualização t-SNE dos dados ELEC2")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
tsne_3d = TSNE(n_components=3, random_state=42)
X_tsne = tsne_3d.fit_transform(X_df)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=Y_df, cmap='viridis', s=50)
fig.colorbar(scatter)
ax.set_title("Visualização t-SNE dos dados ELEC2 (3D)")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")
plt.show()

### UMAP

In [ ]:
umap_2d = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_2d.fit_transform(X_df)

In [ ]:
# Plotando os dados reduzidos com t-SNE
plt.figure(figsize=(8,6))
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=Y_df, cmap='viridis', s=50)
plt.colorbar()
plt.title("Visualização UMAP dos dados ELEC2")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
umap_3d = umap.UMAP(n_components=3, random_state=42)
X_umap = umap_3d.fit_transform(X_df)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_umap[:, 0], X_umap[:, 1], X_umap[:, 2], c=Y_df, cmap='viridis', s=50)
fig.colorbar(scatter)
ax.set_title("Visualização UMAP dos dados ELEC2 (3D)")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")
plt.show()

### PCA

In [ ]:
pca_2d = PCA(n_components=2)
X_pca = pca_2d.fit_transform(X_df)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=Y_df, cmap='viridis')
plt.xlabel('Primeiro Componente Principal')
plt.ylabel('Segundo Componente Principal')
plt.title('Visualização PCA dos dados ELEC2')
plt.colorbar(label='Classes')
plt.show()

In [ ]:
pca_3d = PCA(n_components=3)
X_pca = pca_3d.fit_transform(X_df)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=Y_df, cmap='viridis', s=50)
fig.colorbar(scatter)
ax.set_title("Visualização PCA dos dados ELEC2 (3D)")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")
plt.show()